In [1]:
import pyspark
from pyspark import SparkContext

sc = SparkContext(master='local', appName = 'Titanic')

In [2]:
import findspark
findspark.init()
import pyspark
findspark.find()

'/Users/vunguyen/server/spark-2.4.6-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
'''
conf= pyspark.SparkConf().setAppName('tets_app').setMaster('local')
sc=pyspark.SparkContext(conf=conf)
sc
'''

#sc= SparkContext()
spark = (SparkSession
         .builder
         .appName('Titanic')
         .getOrCreate())
spark

In [4]:
df = spark.read.csv('File/titanic.csv', header = True, inferSchema=True)

In [5]:
df.count()

891

In [6]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [7]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [8]:
my_cols = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [9]:
my_final_data = my_cols.na.drop()

In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [11]:
gender_indexer = StringIndexer(inputCol = 'Sex', outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol = 'SexIndex', outputCol='SexVec')

In [12]:
embark_indexer = StringIndexer(inputCol = 'Embarked', outputCol='EmbarkedIndex')
embark_encoder = OneHotEncoder(inputCol = 'EmbarkedIndex', outputCol='EmbarkedVec')

In [13]:
assembler = VectorAssembler(
            inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkedVec'],
            outputCol='features')

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [15]:
log_reg_titanic = LogisticRegression(featuresCol = 'features',
                     labelCol = 'Survived')

In [16]:
pipeline = Pipeline(stages=[gender_indexer, embark_indexer, 
                            gender_encoder, embark_encoder, 
                            assembler, log_reg_titanic])

In [17]:
train_data,test_data = my_final_data.randomSplit([0.7,0.3])

In [18]:
fit_model = pipeline.fit(train_data)

In [19]:
results = fit_model.transform(test_data)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol ='prediction',
                                       labelCol='Survived')

In [22]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [23]:
AUV = my_eval.evaluate(results)

In [24]:
AUV

0.7830623306233062